In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields)

In [3]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [4]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  811.0


In [5]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  220.0


In [6]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [7]:
print('Huelva (capital)\n')
print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Confirmados PCR 7 días x 1,000 habitantes:  0.11833
Un positivo PCR cada 8451 personas en los últimos 7 días 

Confirmados PCR 14 días x 1,000 habitantes:  0.21578
Un positivo PCR cada 4634 personas en los últimos 14 días


In [8]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [9]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [10]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [11]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [12]:
# Resultado ordenado por Lugar de residencia
tabla

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Aljaraque,46.0,3.0,15.0,21260.0,14.111007,70.555033,NaN
Almonte,41.0,8.0,10.0,24191.0,33.070150,41.337688,NaN
Ayamonte,15.0,3.0,5.0,20946.0,14.322544,23.870906,NaN
Bollullos Par del Condado,18.0,6.0,7.0,14272.0,42.040359,49.047085,NaN
Cabezas Rubias,2.0,1.0,2.0,730.0,136.986301,273.972603,NaN
Condado-Campiña,238.0,47.0,75.0,155057.0,30.311434,48.369309,NaN
Cortegana,8.0,4.0,4.0,4666.0,85.726532,85.726532,NaN
Escacena del Campo,9.0,5.0,8.0,2284.0,218.914186,350.262697,NaN
Gibraleón,12.0,1.0,2.0,12607.0,7.932101,15.864202,NaN


In [14]:
# Save to csv
tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Villalba del Alcor,27.0,17.0,25.0,3338.0,509.286998,748.951468,NaN
Escacena del Campo,9.0,5.0,8.0,2284.0,218.914186,350.262697,NaN
Cabezas Rubias,2.0,1.0,2.0,730.0,136.986301,273.972603,NaN
Minas de Riotinto,14.0,2.0,10.0,3848.0,51.975052,259.875260,NaN
Zalamea la Real,14.0,1.0,3.0,3068.0,32.594524,97.783572,NaN
Isla Cristina,32.0,15.0,20.0,21264.0,70.541761,94.055681,NaN
Cortegana,8.0,4.0,4.0,4666.0,85.726532,85.726532,NaN
Lepe,55.0,6.0,20.0,27431.0,21.873063,72.910211,NaN
Aljaraque,46.0,3.0,15.0,21260.0,14.111007,70.555033,NaN
